# 2015-11-27



Timestamp | Carrier frequency [MHz] | Antenna | Comment
----------|-------------------------|---------|--------
2015-11-27 @ 16:00 UTC | 137.9141; 138.0101 | 10cm whip | Reception indoors using *Gqrx*. Hopefully the former frequency is the carrier of the NOAA-18 APT transmission. **NOTE**: one of the carriers was probably the signal created by the rtl-sdr dongle on its center frequency.
2015-11-27 @ 16:40 UTC | 138.0090            | 10cm whip | Single carrier with no sidebands during NOAA-15 pass. **NOTE:** sorry, no luck, carrier did not go away after the satellite passed.



# 2015-12-2

Setting up equipment for reception and some link budget calculation. Started designing the RF front-end filter for the band.

## Gear

This is only an attemp to hopefully see the downlink transmission on the spectrum analyzer. A Rhode & Schwarz FSH 4 portable spectrum analyzer used as the "receiver". The recepetion attempt was made using a is a log-periogic antenna

**TODO:** get the spec for the antenna

## Contacts

### Two failed attempts

The first two attempts at detecting the APT transmission using the FSH 4 and a log-periodic antenna failed.

Reception configuration:

- Spectrum analyzer as receiver
    - center: 137.5MHz
    - span: 1MHz
    - RBW: 3kHz
    - Att: 0 dB
- antenna: wideband log-periodic

The RBW setting might have been something else, I'm not sure. In retrospect, considering the 34 kHz width an APT transmission, the spectrum analyser setting might have not been suitable. The satellite orbits were not great either, and we were not exaclty sure in which direction south was.

**EDIT:** it is likely that the 137.5 MHz fequency was not correct, since the satellites use different downlink frequencies.

### 16:27 UTC+2 NOAA 15 orbit 91282 137.5000 MHz

This attempt seemed promising, with an orbit peak elevation of over 70 degrees, which is higher than the two earlier attemps. But, no luck.

We also tried another antenna (**TODO:** what antenna?) but saw no difference.

Reception configuration:

- FSH 4 Spectrum analyzer as receiver
    - center: 137.5 MHZ
    - span: 200 kHz
    - RBW: 3 kHz
    - Att: 0 dB
- antenna: wideband log-periodic


### 16:50 UTC+2 NOAA 18 orbit 54283 137.9125 MHz

**successfully viewed the transmission** on the spectrum analyzer. 

Reception configuration:

- FSH 4 Spectrum analyzer as receiver
    - center: 137.9125 MHZ
    - span: 100 kHz
    - RBW: 100 Hz
    - Att: 0dB
    - ref-level: -60 dBm
- antenna: wideband log-periodic
- no RF filters or amplifiers


Photo of the spctrum analyzer screen during the peak signal strength of the pass:
<img src="./photos/20151202_170044.jpg" alt="ATP reception spectur" width="400"/>

In [1]:
import math
noise_power_100Hz = 10 * math.log(138e-23 * 290 * 100, 10)  # P = kTB
noise_power_3kHz =  10 * math.log(138e-23 * 290 * 3000,10)
print(noise_power_100Hz)
print(noise_power_3kHz)

-163.977229157
-149.20601661


# 2015-12-3


## 16:40 UTC+2 NOAA18 orbit 54297 137.9125 MHz

A semi-successful test. The RTL-SDR definitely received something, but there is seems to be a signal-nosie ration problem when trying to decode the image from the baseband recording


- FSH 4 Spectrum analyzer
    - center: 137.9125 MHz
    - span: 100 kHz
    - RBW: 100 Hz
    - Att: 0 dB
    - ref-level: -60dB
- RTL-SRD dongle
    - demodulation: narrow FM
    - software filter: 40kHz
    
![screenshot of gqrx + gnome predict](./photos/test-screenshots/Screenshot from 2015-12-03 16:46:28.png)
    

# 2015-12-08

Got some coax connectors and copper board for building the band pass filter. Measured the inductance of some adjustable RF-inductors.

# 2015-12-14

Three reception attemps using log-periodic antenna (tests 6, 7, and 8) and RTL dongle. Tried saving both raw IQ data as well as narrow-FM demodulated data, although gqrx seems not to make any diffenerence between the two.

After trying to decode all of the recordings with wxtoimg, there are no successuful results.

# 2016-02-10

Built a simple bandpass filter using an adjustable capacitance tapped LC circuit. The amplitude response was ok frequency wise, but the passband attenuation was about 25 dB for some reason.

Turns out the coax connectors used had S-connectors, which are rated for 75 $\Omega$, not 50 $\Omega$ for which the both the circuit and the circuit analyzer are rated.

**Update**: The coax leads were about 15 cm long. Having measured the loss to be less than 0.5 dB, the conclusion is that the error is in the circuit.

<img alt="photo of the filter amplitude response. It has 25 dB passband attenuation" src="./photos/20160211_133212.jpg" width="400" />


<img src="./photos/20160211_133454.jpg" alt="Coax S-connector" width ="400"/>

# 2016-02-11

Spent a bunch of time looking into quadfilar helix antennas. Found some dimension calculators online and build instructions online:

- http://www.jcoppens.com/ant/qfh/calc.en.php
- http://homepage.ntlworld.com/phqfh1/qfh_diy_guide.htm
- http://www.housedillon.com/?p=1461

## Dimension from jcoppens caclulator

Larger loop

element                           | larger  | smaller
----------------------------------|---------| 
Total length                      | 2.40 m | 02.28 m
Vertical separator                | 0.88 m | 0.84 m
Total compensated length          | 2.41 m | 2.29 m
Antenna height                    | 0.73 m | 0.69 m
Internal diameter                 | 0.31 m | 0.30 m
Horizontal separator              | 0.32 m | 0.30 m




- Design frequency:
	- Evident?
- Number of turns (twist):
	- What's the twist of the antenna? (normally 0.5 (180 degrees)
- Length of one turn:
	- A few variations of the antenna exist. Normally the circumference (length of the loop) is 1 wavelength, but 1.5 wavelength and 2 wavelength versions exist.
- Bending diameter:
	- As it's impossible to bend the corner abruptly at 90 degrees, this value is needed for the calculations. It's measured from the bending center to the center of the tube.
- Conductor diameter:
	- External diameter of the tube or coax cable.
- Diameter/height ratio:
	- Most frequently this ratio is 0.44, but slightly lower values (0.3 to 0.4) give better horizon coverage.
- Wavelength:
	- Wavelength, corresponding to the selected frequency.
- Compensated wavelength:
	- Wavelength, compensated according to the conductor diameter.
- Bending correction:
	- Correction value needed according to the bending diameter.
- Total length:
	- Total length of the loop, before compensation.
- Total loop cmpensated length:
	- Total length of the loop, compensated for the bending effect, and the fact that the loop must be slightly larger (or smaller). This is the amount of tubing necessary for this loop.
- Compensated vertical separation:
	- Vertical separation (without the 'bends').
- Compensated horizontal separation:
	- This is in fact the horizontal part without the 'bends', and corresponds to the horizontal pipe necessary to support the cable.
- Antenna height:
	- Height of the loop (twisted!).
- Internal diameter:
	- The diameter of the (imaginary) cylinder on which the loop would be wound. 

# 2016-03-02

Designed three different bandpass circuits using AWR. They capacitance tappe circuits with either 1, 2, or 3 LC tank circiuts

See the [/circuits/bandpass-filter-AWR/](./circuits/bandpass-filter-AWR/) directory the AWR CAD files and screenshots of the bandpass filter circuit and their analysis results.

# 2016-03-10

Wrote and refactored a bunch of documentaition and cleaned some files. 

Learned how to use `wxtoimg` from the commandline. 

```
wxtoimg -tn -N some.wav > some.wav.jpg
```

The `-tn` options sets the satellite type to NOAA. `-N` specifies a northbound pass. The passes that happen around 1600 UTC+2 are all northbound.

Decoded images from all the reception attempts can be found in the [/recodings](./recordings) directory. As of yet, non of the recordings decode properly, they are mostly noise. Below is an example of a decoded image. The gap between the two sensor images is visible when viewing the image at native resolution.

![whitenoise image decoded from ATP transmission](./photos/gqrx-20151203-144531.wav.wxtoimg.jpg)

# 2016-03-11

Wrote some python DSP code [in the DSP notebook](./signal-processing/APT-DSP.ipynb)

The test images procduced by the script contain a gradient in Video A and random data in Video B 


## The data frequency higher than carrier

Each line is 2080 pixels wide, and and the duration af the line is 0.5 seconds. The lowest frequency in the data is therfore the symbol rate 4160 Hz. In the APT system, the image data is modulated using AM modulation with a modulation index of 94 % with a 2400 Hz carrier. 

**TODO:** what does the spectrum look like ?
**TODO:** how well does it modulate into a 34 kHz FM transmission

## AM demodulating

An AM carrier can be demodulated by multipying it with a sine wave, but that sine wave must be 180 off-phase from the carrier, have the same amplitude as the unmodulated carrier, and the same **exact** frequency. These three requirements make it somewhat complex to implement

If the carrier frecuency $f_c$ and the demodulation frequency are not the same, it will cause a phase-shift over time, essentially boosting the carrier at times, and and demodulating the carrier at others.

---

An AM carrier can be easily demodulated if the carrier frequency is know. By sampling the waveform at 4 times the carrier frequency, we can take any two consecutive samples and get the carrier amplitude at that moment in time.

This is essentially the length of a complex vector:

$A = \sqrt{s_1^2 + s:2^2}$

where $s_1$ and $s_2$ are two consecutive samples. If it's not obvious, try to draw a sine wave and do the math using any two samples separated by 90°. This will be true for an two consecutive points

## ATP data signal waveform

Below are two graphs generated with the DSP notebook. One depicts the waveform of a single ATP line, while the other depicts a more detailed view of the middle of the first waveform.

!["graph of APT-line-waveform-overview"](./photos/APT-line-waveform-overview.svg)
!["graph of APT-line-waveform-video-transition"](./photos/APT-line-waveform-video-transition.svg)


## 23:26 Image decoding sucessful


The .png below was produced by supplying a recoded .wav to a custom GNU radio python script.


!["The first test image decoded using GNU radio"](./photos/first-decoded-test-image.png)
!["first sucessfully decoded image from satellite. Using GNU radio"](./photos/first-decoded-satellite-image.png)




# 2016-03-12

Tried to receive NOAA 19 ((orbit 36562) with a simple $\lambda / 4 $monopole. I did not have access to a rooftop, so I tried receiveing the transmission from a parking lot between buildings. The satellite signal did not appear in the spectrum view in gqrx. NOAA19 might have been moved to 137.6 MHz from 137.5 MHz, where I was listening.

----

See [/orbit-data/orbit-plot.ipynb](./orbit-data/orbit-plot.ipynb) for a notebook where I've made some plots of the elevation/azimuth data of the orbits I've attempted to receive images from.


# 2016-03-12

Did a bunch of antenna work, and compared how well the antennas worked. Documented in the [Antenna comparisons section if notes.ipynb](./notes.ipynb#Antenna-comparison-tests)

Also did some input matching measurements of the RTL-SDR

# 2016-03-22

Took a closer look at the RTL-SDR dongle. Turns out the first stage the RF signal is fed to is an R820T, not the RTL2932U. RTL-SDR measurements in [notes.ipynb](./notes.ipynb) will be compared the what is in the datasheet.

Reading through the datasheet gave some insights as to who some of the charasteristics of the tuner/mixer circuits are are measured modelled. E.g I had not seen phase noise given as dBc/Hz @ some frequency before, altough in hindsigth is seems like a very reasonable unit.  


# 2016-03-23

Read about geosataionary orbits and looked for other systems similar to the NOAA POES constellation. Unfortunately NOAA GOES geostationary satellites are sationary over the americas, and therefore not available for reception in Finland.

The EUMETSAT program has interesing wheather and science satellites in goestationary orbits above Africa, and in polar orbits. 

# Time overview

In [ ]:

import datetime
import matplotlib.pyplot as plt
time = [
    [16.00, 20.00, datetime.date(2015,11,30)], 
    [16.00, 20.00, datetime.date(2015,11,29)],
    [16.00, 20.00, datetime.date(2015,11,28)], # thes fors ones are just some guesses
    [16.00, 20.00, datetime.date(2015,12,1 )], # although these later ones are not the accurate either
    [09.00, 17.50, datetime.date(2015,12,2 )], # Whoo! first contact
    [19.00, 20.00, datetime.date(2015,12,2 )], # writing documentation about first contact
    [08.00, 11.00, datetime.date(2015,12,3 )], # designing df band pass filter
    [11.30, 15.00, datetime.date(2015,12,3 )], # calculating coil sizes and stuff
    [15.00, 17.00, datetime.date(2015,12,3 )], # prepping for test no. 5
    [12.00, 13.40, datetime.date(2015,12,3 )], # bandpass filter build techniqe planning 
    [15.50, 18.50, datetime.date(2015,12,14)], # tests 6,7,8, and trying to decode recordings
    [14.50, 16.00, datetime.date(2016, 2,8 )], # Simulating bandpassfilter using AWR 
    [09.00, 12.00, datetime.date(2016, 2,10)], # built bandpass filter prototype, dealt with caax connectors
    [12.50, 13.70, datetime.date(2016, 2,11)], # competed first bandpass protopyte, analyzed it. bad results
    [15.50, 16.50, datetime.date(2016, 2,11)], # looking into antennas, reading existing documentation
    [18.30, 19.75, datetime.date(2016, 2,11)], # looking into antennas, palning on building QFH
    [ 9.00, 12.15, datetime.date(2016, 3,2 )], # after the bad results from the earlier prototype, made new simulations using AVR
    [14.00, 19.00, datetime.date(2016, 3,10)], # improved documentation and organized files. learned ot use wxtoim commandline
    [0.0,   6.00 , datetime.date(2016, 3,11)], # python DSP code for generating APT baseband signals
    [7.0,   9.0  , datetime.date(2016, 3,11)], # antenna stuff + link budget
    [13.0,  16.00, datetime.date(2016, 3,11)], # DSP stuff for generating APT data signals
    [16.00, 17.5 , datetime.date(2016, 3,11)], # generate bitmaps from APT data signals
    [18.25, 22.0 , datetime.date(2016, 3,11)], # DSP can now generate WAV files. wxtoim almost decodes them
    [22.30, 23.50, datetime.date(2016, 3,11)], # Designed a simple APT decoder in gnuradio. It sucessfully decoded the first image
    [08.50, 10.00, datetime.date(2016, 3,12)], # try to use the RTL-SDR block in DRC
    [10.00, 11.33, datetime.date(2016, 3,12)], # tried a few monopoles on 130.05 MHz, plannin field test 
    [14.00, 14.30, datetime.date(2016, 3,12)], # failed to receive form NOAA 19 using λ/4 monopole
    [15.50, 16.00, datetime.date(2016, 3,12)], # decoded all satellite recordings into images
    [19.00, 21.00, datetime.date(2016, 3,12)], # Draw orbit data plots
    [09.00, 11.00, datetime.date(2016, 3,16)], # Reading about constructing HIGH-Q inductors
    [11.00, 13.50, datetime.date(2016, 3,16)], # Antenna construction (di- and monople) with bnc connectos
    [14.00, 16.50, datetime.date(2016, 3,16)], # rtl input S11 reflection measurements
    [16.50, 17.25, datetime.date(2016, 3,16)], # reception of NOAA 18 using dipole antenna
    [18.00, 18.50, datetime.date(2016,3,16)], # Refining notes and documentation from the day
    #[# # ],
    [09.00, 13.00, datetime.date(2016,3,20)], # documenting signal processing, look into post processing
    [16.00, 17.25, datetime.date(2016,3,22)], # searching/reading deepinfo/datasheets for RTL-SDR
    [10.00, 12.25, datetime.date(2016,3,25)], # looking into GOES, METEOSAT and other systems
]

start = [row[0] for row in time]
end =  [row[1] for row in time]
len = [row[1] - row[0] for row in time]
print sum(len)
plt.bar( [row[2] for row in time],len)
plt.show()


93.73
